# TOUGH3 postprocessing

- Reads output generated from the TOUGH3-FLAC3D project 2dldV6 and plots the pressure evolution in time at the injection point to evaluate the difference/effect of stress-independent permeability with stress-dependent permeability.
- In other words, it evaluates the effect of the coupling with FLAC3D and without

In [1]:
# Import the required libraries
import toughio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
from matplotlib import gridspec

import matplotlib
matplotlib.use('Qt5Agg')

# setting up for plotting
plt.rcParams['figure.dpi'] = 50
plt.rcParams['font.size'] = 25
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'

### Execute for both examples 2dldV6 and 2DSEP5V6

-> The following cell introduces the routines to read the output from TOUGH as it is

In [2]:
# Read the output produced by TOUGH3

output_path = '/home/victor/Desktop/toughio/examples/TOUGH3-FLAC3D_examples/2dldV6/'
folder = ['2/', '3/']
# first_cell = ['             A11 0', '             A1152']
output_file = 'OUTPUT_ELEME.csv'

sim_data = []

for s in range(len(folder)):
    output_filepath = output_path + folder[s] + output_file
    
    ele_raw = pd.read_csv(output_filepath, index_col=False, usecols=[0])
    ids_ele1 = ele_raw[ele_raw.values == '             A11 0'].index.values                                                 # 13 empty spaces before element ID, then 5 characters for element ID -> if chenges, find out with len(ele_raw.values[X,0])
    # ids_ele1 = ele_raw[ele_raw.values == first_cell[s]].index.values
    # ids_ele1 = ele_raw[ele_raw.values == '             A2  1'].index.values
    ids_t = ids_ele1 - 1
    times_raw = ele_raw.values[ids_t, 0]
    times_corr = []
    [times_corr.append(float(times_raw[i][11:])) for i in range(len(times_raw))]                                            # in seconds
    # [times_corr.append(float(times_raw[i].strip())) for i in range(len(times_raw))]
    
    # print output times:
    print('Found output at the following times:', )
    [print('TIME [years]: ' + str(int(times_corr[i]/(60*60*24*365.25)))) for i in range(len(times_corr))]
    
    data = []
    header_names_raw = pd.read_csv(output_filepath, nrows=0).columns
    header_names = pd.Index([element.strip() for element in header_names_raw])
    for j in range(len(ids_t)):
        if j==0:    # only works if there are more than just one time output (!)
            df = pd.read_csv(output_filepath, header=None, index_col=False, skiprows=list(range(0, ids_t[j]+2)), nrows=((ids_t[j+1]-1)-ids_t[j]))
        elif j==(len(ids_t)-1):
            df = pd.read_csv(output_filepath, header=None, index_col=False, skiprows=list(range(0, ids_t[j]+2)), nrows=((len(ele_raw.values)-1)-ids_t[j]))
        else:
            df = pd.read_csv(output_filepath, header=None, index_col=False, skiprows=list(range(0, ids_t[j]+2)), nrows=((ids_t[j+1]-1)-ids_t[j]))
        df = df.reset_index(drop=True)
        # added the next two lines:
        df[0] = df[0].apply(lambda x: x.strip())
        df.columns = header_names
        df = df.set_index(df.columns[0])
        data.append(df)
    sim_data.append(data)
    

Found output at the following times:
TIME [years]: 0
TIME [years]: 0
TIME [years]: 0
TIME [years]: 0
TIME [years]: 2
TIME [years]: 5
TIME [years]: 9
TIME [years]: 9
Found output at the following times:
TIME [years]: 0
TIME [years]: 0
TIME [years]: 0
TIME [years]: 0
TIME [years]: 2
TIME [years]: 5
TIME [years]: 9


### Execute only for 2dldV6

-> This routine reproduces Fig. 2 from Rutqvist and Tsang (2022) and Fig. 5.6 from the TOUGH-FLAC python-package manual

In [3]:
# Collect the necessary data for plotting

inj_cell = 'A1793'  # injection cell 2dldV6

pres_vals = np.zeros((len(folder), len(ids_t)))

for s in range(len(folder)):
    for j in range(len(ids_t)):
        pres_vals[s, j] = sim_data[s][j].PRES[inj_cell]

In [5]:
# Plotting 1

fig = plt.figure(figsize=[12, 12])
gs = gridspec.GridSpec(nrows=1, ncols=1)
ax1 = fig.add_subplot(gs[0])

colors = ['tab:red', 'tab:blue']

for s in range(len(folder)):
    ax1.plot(np.asarray(times_corr) / (24.0 * 365.0 * 3600.0), pres_vals[s,:] / (1.e6), c=colors[s], ls='-', lw=2)
    
ax1.set_xlabel('Time [years]')
ax1.set_ylabel('Pressure at injection point [MPa]')

fig.show()
fig.savefig('2dldV6.svg')

In [ ]:
# Plotting 2



### Execute only for 2DSEP5V6

-> This routine reproduces Fig. 5.11 from the TOUGH-FLAC python-package manual